In [1]:
# Setup API
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("RB_Vehicle_1")


In [2]:
# Setup môi trường
# Cài đặt các gói, NHƯNG ép numpy phải ở phiên bản 1.x
!pip install "rfdetr[metrics]" roboflow "numpy<2.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# Check gpu
!nvidia-smi

Sat Oct 25 09:17:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   36C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
# Tải dataset từ roboflow
from roboflow import Roboflow
rf = Roboflow(api_key=secret_value_0) # API key
project = rf.workspace("myworkspace-jlyfn").project("vehicle-detection-bnyxm-lbeef")
version = project.version(1)
dataset = version.download("coco")

DATASET_PATH = dataset.location
print(f"Dataset downloaded to: {DATASET_PATH}")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Vehicle-Detection-1 in coco:: 100%|██████████| 1099/1099 [00:00<00:00, 5817.44it/s]


Dataset downloaded to: /kaggle/working/Vehicle-Detection-1


In [5]:
from rfdetr import RFDETRBase

OUTPUT_PATH = "/kaggle/working/RFDETR_Training"

model = RFDETRBase()

model.train(
    dataset_dir = DATASET_PATH,
    output_dir = OUTPUT_PATH,
    epochs=100,
    batch_size = 4,
    grad_accum_steps=4,
    lr=1e-4,
    tensorboard=True
)

print(f"Complete! Checkpoints saved at: {OUTPUT_PATH}")

2025-10-25 09:18:23.564561: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761383903.912399      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761383904.019169      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
rf-detr-base.pth: 100%|██████████| 355M/355M [00:03<00:00, 102MiB/s]


Loading pretrain weights
TensorBoard logging initialized. To monitor logs, use 'tensorboard --logdir /kaggle/working/RFDETR_Training' and open http://localhost:6006/ in browser.
Not using distributed mode
git:
  sha: N/A, status: clean, branch: N/A

Namespace(num_classes=20, grad_accum_steps=4, amp=True, lr=0.0001, lr_encoder=0.00015, batch_size=4, weight_decay=0.0001, epochs=100, lr_drop=100, clip_max_norm=0.1, lr_vit_layer_decay=0.8, lr_component_decay=0.7, do_benchmark=False, dropout=0, drop_path=0.0, drop_mode='standard', drop_schedule='constant', cutoff_epoch=0, pretrained_encoder=None, pretrain_weights='rf-detr-base.pth', pretrain_exclude_keys=None, pretrain_keys_modify_to_load=None, pretrained_distiller=None, encoder='dinov2_windowed_small', vit_encoder_num_layers=12, window_block_indexes=None, position_embedding='sine', out_feature_indexes=[2, 5, 8, 11], freeze_encoder=False, layer_norm=True, rms_norm=False, backbone_lora=False, force_no_pretrain=False, dec_layers=3, dim_feedfo

fatal: not a git repository (or any of the parent directories): .git


Epoch: [0]  [ 0/47]  eta: 0:06:48  lr: 0.000100  class_error: 100.00  loss: 15.8322 (15.8322)  loss_ce: 4.5419 (4.5419)  loss_bbox: 0.0803 (0.0803)  loss_giou: 0.0377 (0.0377)  loss_ce_0: 3.7728 (3.7728)  loss_bbox_0: 0.1046 (0.1046)  loss_giou_0: 0.0458 (0.0458)  loss_ce_1: 4.4479 (4.4479)  loss_bbox_1: 0.1117 (0.1117)  loss_giou_1: 0.0481 (0.0481)  loss_ce_enc: 2.1287 (2.1287)  loss_bbox_enc: 0.3684 (0.3684)  loss_giou_enc: 0.1446 (0.1446)  loss_ce_unscaled: 4.5419 (4.5419)  class_error_unscaled: 100.0000 (100.0000)  loss_bbox_unscaled: 0.0161 (0.0161)  loss_giou_unscaled: 0.0188 (0.0188)  cardinality_error_unscaled: 3898.7500 (3898.7500)  loss_ce_0_unscaled: 3.7728 (3.7728)  loss_bbox_0_unscaled: 0.0209 (0.0209)  loss_giou_0_unscaled: 0.0229 (0.0229)  cardinality_error_0_unscaled: 3899.0000 (3899.0000)  loss_ce_1_unscaled: 4.4479 (4.4479)  loss_bbox_1_unscaled: 0.0223 (0.0223)  loss_giou_1_unscaled: 0.0240 (0.0240)  cardinality_error_1_unscaled: 3899.0000 (3899.0000)  loss_ce_enc_un